<a href="https://colab.research.google.com/github/mrdbourke/pytorch-deep-learning/blob/main/extras/exercises/05_pytorch_going_modular_exercise_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:** 

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [11]:
%%writefile 05_exercise_scripts/get_data.py

import os
import zipfile

from pathlib import Path

import requests

# setup the data directory
data_dir = Path("data")
image_path = data_dir / "pizza_steak_sushi"

if image_path.is_dir():
    print(f'Image path {image_path} is a directory')
else:
    print(f'Image path {image_path} is not a directory')
    print('creating the directory...')
    image_path.mkdir(parents=True, exist_ok=True)
    
with open(data_dir / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading data...")
    f.write(request.content)
    
with zipfile.ZipFile(data_dir / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping data...")
    zip_ref.extractall(data_dir)
    
os.remove(data_dir / "pizza_steak_sushi.zip")

Overwriting 05_exercise_scripts/get_data.py


In [12]:
# Example running of get_data.py
!python 05_exercise_scripts/get_data.py

Image path data/pizza_steak_sushi is not a directory
creating the directory...
Unzipping data...


## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular). 

In [23]:
%%writefile 05_exercise_scripts/train.py
# YOUR CODE HERE
import os
import torch
from torchvision.transforms import v2 as transforms
import argparse
import data_setup, model_builder, engine, utils

# setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

# setup parser
parser = argparse.ArgumentParser(description="butuh hyperparameter")

# setup argparser untuk num_epochs
parser.add_argument("--num_epochs", type=int, default=10, help="number of epochs to train model, default=10")

# setup argparser untuk batch_size
parser.add_argument("--batch_size", type=int, default=32, help="batch size for training model, default=32")

# setup argparser untuk hidden layer
parser.add_argument("--hidden_units", type=int, default=64, help="hidden layer size for model, default=128")

# setup argparser untuk learning rate
parser.add_argument("--learning_rate", type=float, default=0.001, help="learning rate for model, default=0.001")

# setup argparser untuk training directory
parser.add_argument("--train_dir", type=str, default="data/train", help="training directory for model, default=data/pizza_steak_sushi/train")

# setup argparser untuk testing directory
parser.add_argument("--test_dir", type=str, default="data/test", help="testing directory for model, default=data/pizza_steak_sushi/test")

args = parser.parse_args()

NUM_EPOCHS = args.num_epochs
BATCH_SIZE = args.batch_size
HIDDEN_UNITS = args.hidden_units
LEARNING_RATE = args.learning_rate
print(f'[INFO] Training model for {NUM_EPOCHS} epochs with batch size {BATCH_SIZE}, hidden layer size {HIDDEN_UNITS}, and learning rate {LEARNING_RATE}')

# setup directories
train_dir = args.train_dir
test_dir = args.test_dir

print(f'[INFO] Training directory: {train_dir}')
print(f'[INFO] Testing directory: {test_dir}')

data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToImage(),
    transforms.ToDtype(torch.float32,scale=True),
])

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir, 
                                                                               test_dir, 
                                                                               data_transform, 
                                                                               BATCH_SIZE)

model = model_builder.TinyVGG(input_shape=3 ,hidden_units=HIDDEN_UNITS, output_shape=len(class_names)).to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             device=device,
             epochs=NUM_EPOCHS)

utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_tiny_vgg_script_mode.pth",)

Overwriting 05_exercise_scripts/train.py


In [25]:
# Example running of train.py
!python 05_exercise_scripts/train.py --num_epochs 20 --batch_size 128 --hidden_units 128 --learning_rate 0.0003

[INFO] Training model for 20 epochs with batch size 128, hidden layer size 128, and learning rate 0.0003
[INFO] Training directory: data/train
[INFO] Testing directory: data/test
100%|███████████████████████████████████████████| 20/20 [01:09<00:00,  3.48s/it]
[INFO] Saving model to: models/05_tiny_vgg_script_mode.pth


## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function). 
* You may also have to write code to load in a trained model.

In [34]:
%%writefile 05_exercise_scripts/predict.py
# YOUR CODE HERE
import torch
import torchvision
import argparse

import model_builder

parser = argparse.ArgumentParser(description="butuh hyperparameter")

# setup argparser untuk image_path
parser.add_argument("--image_path", help="path to image file for prediction")

parser.add_argument("--model_path",
                    default="models/05_tiny_vgg_script_mode.pth",
                    type=str,
                    help="path to model for prediction")

args = parser.parse_args()

class_names = ["pizza", "steak", "sushi"]

device = "cuda" if torch.cuda.is_available() else "cpu"

IMG_PATH = args.image_path
print(f'[INFO] Image path: {IMG_PATH}')

def load_model(filepath=args.model_path):
    model = model_builder.TinyVGG(input_shape=3, 
                                  hidden_units=128, 
                                  output_shape=3).to(device)
    model.load_state_dict(torch.load(filepath))
    
    return model

def predict_on_image(image_path=IMG_PATH, filepath=args.model_path):
    model = load_model(filepath)
    
    image = torchvision.io.read_image(str(IMG_PATH)).type(torch.float32)
    
    image = image / 255.0
    
    transform = torchvision.transforms.Resize((64, 64))
    image = transform(image)
    
    model.eval()
    with torch.inference_mode():
        image = image.to(device)
        pred_logits = model(image.unsqueeze(dim=0))
        pred_probs = torch.softmax(pred_logits, dim=1)
        
        pred_label = torch.argmax(pred_logits, dim=1)
        pred_label_class = class_names[pred_label]
        
    print(f'[INFO] Pred Class: {pred_label_class}, pred prob: {pred_probs.max():.3f}')
    
if __name__ == "__main__":
    predict_on_image()

Overwriting 05_exercise_scripts/predict.py


In [35]:
# Example running of predict.py 
!python 05_exercise_scripts/predict.py --image data/test/sushi/175783.jpg

[INFO] Image path: data/test/sushi/175783.jpg
/root/ZTM_Pytorch/05_exercise_scripts/predict.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.